## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-06-18-51-01 +0000,nyt,Judge Poised to Free Abrego Garcia if Official...,https://www.nytimes.com/2025/10/06/us/politics...
1,2025-10-06-18-49-44 +0000,nyt,"Trump Gave Partisan Speech to Navy Troops, as ...",https://www.nytimes.com/2025/10/06/us/politics...
2,2025-10-06-18-49-33 +0000,nyt,"What Is the National Guard, and Who Controls It?",https://www.nytimes.com/2025/10/06/us/what-is-...
3,2025-10-06-18-48-34 +0000,nyt,Trump Says Portland Is ‘Burning.’ Here’s What ...,https://www.nytimes.com/live/2025/10/06/us/nat...
4,2025-10-06-18-48-28 +0000,nypost,Judge upholds Nassau County’s ban on transgend...,https://nypost.com/2025/10/06/us-news/judge-up...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2307/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,41
56,new,17
86,gaza,15
0,judge,13
297,government,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
13,2025-10-06-18-34-32 +0000,nypost,Trump pushes for his Gaza peace deal to be put...,https://nypost.com/2025/10/06/us-news/trump-pu...,97
126,2025-10-06-13-03-45 +0000,nypost,Inside Trump’s blunt exchange with Netanyahu a...,https://nypost.com/2025/10/06/world-news/insid...,88
36,2025-10-06-17-52-24 +0000,nypost,Retired IDF general who saved his family on Oc...,https://nypost.com/2025/10/06/world-news/retir...,86
224,2025-10-05-23-53-33 +0000,bbc,Trump urges mediators to 'move fast' as key Ga...,https://www.bbc.com/news/articles/cj3y6g43248o...,85
183,2025-10-06-08-50-56 +0000,wapo,Judge deals Trump new setback in plans to depl...,https://www.washingtonpost.com/nation/2025/10/...,85


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
13,97,2025-10-06-18-34-32 +0000,nypost,Trump pushes for his Gaza peace deal to be put...,https://nypost.com/2025/10/06/us-news/trump-pu...
35,63,2025-10-06-17-52-46 +0000,nyt,"Paramount Buys Bari Weiss’s Free Press, Starti...",https://www.nytimes.com/2025/10/06/business/me...
112,46,2025-10-06-14-04-48 +0000,nypost,Supreme Court rejects Ghislaine Maxwell’s appe...,https://nypost.com/2025/10/06/us-news/supreme-...
239,38,2025-10-05-21-24-00 +0000,wsj,The federal government shutdown is dragging to...,https://www.wsj.com/politics/policy/government...
123,37,2025-10-06-13-21-45 +0000,nyt,Judge Blocks National Guard Deployment in Oreg...,https://www.nytimes.com/2025/10/05/us/trump-na...
163,33,2025-10-06-10-00-00 +0000,nypost,"These Americans died fighting in Ukraine, now ...",https://nypost.com/2025/10/06/world-news/paren...
229,31,2025-10-05-23-07-07 +0000,bbc,Gisèle Pelicot to face one of her rapists in c...,https://www.bbc.com/news/articles/cdx2d9lynk1o...
70,30,2025-10-06-16-21-57 +0000,nypost,Eric Adams jets off to Albania after dropping ...,https://nypost.com/2025/10/06/us-news/eric-ada...
159,30,2025-10-06-10-07-35 +0000,cbc,"American, Japanese scientists win Nobel prize ...",https://www.cbc.ca/news/world/nobel-sweden-med...
11,29,2025-10-06-18-37-00 +0000,wsj,Treasury Secretary Scott Bessent has appointed...,https://www.wsj.com/politics/policy/bessent-pi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
